In [ ]:
# Author: David Greene
# The purpose of this program is to pull data from a database hosted in MongoDB and
# to modify that data as needed by a user with the correct credentials. Said data will then
# be manipulated to be displayed in a format that any user can understand.

# Imports Dash libraries and core components
# Imports pandas, pymongo, and mongo for database manipulation

from jupyter_plotly_dash import JupyterDash

import base64
import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

# Imports CRUD Python module file and class
from animal_shelter import AnimalShelter


###########################
# Data Manipulation / Model
###########################
##### Inputs username and password and CRUD Python module name to gain access
username = "aacuser"
password = "davidGreene"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#### Adds Grazioso Salvare’s logo
image_filename = 'GraziosoSalvareLogo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#### Basic designs of the layout including imported image format
app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Hr(),
    html.Div(
# Adds in code for the interactive filtering options.
        dcc.RadioItems(
            id='filter-type',
            # created the labels and keys based on the Grazioso requirements
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster Rescue ', 'value': 'Disaster Rescue or Individual Tracking'},
                {'label': 'Age Trends', 'value': 'Age Trends'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
            labelStyle={'display': 'inline-block'}
        ),
        
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
##### Sets up the features for the interactive data table to make it user-friendly for the client
        style_cell={'textAlign': 'left'},
        sort_action='native',
        style_header={
            'backgroundColor': 'rgb(30, 30, 30)',
            'color': 'white'
        },
        style_data={
            'backgroundColor': 'rgb(50, 50, 50)',
            'color': 'white'
        },
    ),
    html.Br(),
    html.Hr(),
# This sets up the dashboard so that the table and the geolocation chart are next to each other
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ]),
    html.H4("David Greene")
])

#############################################
# Interaction Between Components / Controller
#############################################
    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(value):
# Code to filter interactive data table with MongoDB queries
    
    df = pd.DataFrame.from_records(shelter.read({}))
    
    # adjusts the read request for the desired dog type and status, WATER RESCUE, based on Grazioso requirements
    if value == 'Water Rescue':
        df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog","breed":{"$in":["Labrador Retriever Mix",
                                                    "Chesapeake Bay Retriever","Newfoundland"]},
                                                    "sex_upon_outcome":"Intact Female",
                                                    "age_upon_outcome_in_weeks":{"$gte":26},
                                                    "age_upon_outcome_in_weeks":{"$lte":156}}))
        
    # adjusts the read request for the desired dog type and status, MOUNTAIN/WILDERNESS RESCUE, based on Grazioso requirements
    if value == 'Mountain or Wilderness Rescue':
        df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog","breed":{"$in":["German Sheperd",
                                                    "Alaskan Malamute","Old English Sheepdog","Siberian Husky",
                                                    "Rottweiler"]}, "sex_upon_outcome":"Intact Male",
                                                    "age_upon_outcome_in_weeks":{"$gte":26},
                                                    "age_upon_outcome_in_weeks":{"$lte":156}}))
        
    # adjusts the read request for the desired dog type and status, DISASTER RESCUE/TRACKING, based on Grazioso requirements
    if value == 'Disaster Rescue or Individual Tracking':
        df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog","breed":{"$in":["Doberman Pinscher",
                                                    "German Sheperd","Golden Retriever","Bloodhound",
                                                    "Rottweiler"]}, "sex_upon_outcome":"Intact Male",
                                                    "age_upon_outcome_in_weeks":{"$gte":20},
                                                    "age_upon_outcome_in_weeks":{"$lte":300}}))
    
    # adjusts the read request for data mined trends for desired dogs, AGE TRENDS, based on Grazioso requirements
    if value == 'Age Trends':
        df = pd.DataFrame.from_records(shelter.read({"animal_type":"Dog", "age_upon_outcome":{"$lte":3}}))
        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
    data=df.to_dict('records')
            
    return (data,columns)


# Highlights user selected columns
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


#function to update the pie chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    # imports the currently displayed data
    dff = pd.DataFrame.from_dict(viewData)
    #creates the values needed for the names (breed) and values (recurring counts)
    names = dff['age_upon_outcome'].value_counts().keys().tolist()
    values = dff['age_upon_outcome'].value_counts().tolist()
    #creates a pie chart based on the data above
    return [
        dcc.Graph(            
            figure = px.pie(
                data_frame=dff, 
                values = values, 
                names = names, 
                color_discrete_sequence=px.colors.sequential.RdBu,
                width=800, 
                height=500   
            )
        )
    ]


@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
##### Code for your geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff.iloc[0,13],dff.iloc[0,14]], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[0,9])
                ])
            ])
        ])
    ]


app